In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict 
import itertools

In [28]:
P_list = ["dummy", 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.92, 0.94]
# df = pd.DataFrame(data = np.array([[0, 1, 1, 0, 0, 0, 0, 0],
#                                    [0, 0, 0, 1, 1, 0, 0, 0],
#                                    [0, 0, 0, 1, 0, 1, 0, 1],
#                                    [0, 0, 0, 0, 1, 1, 0, 1],
#                                    [0, 0, 0, 0, 0, 1, 1, 0],
#                                    [0, 0, 0, 0, 0, 0, 1, 1],
#                                    [0, 0, 0, 0, 0, 0, 0, 0],
#                                    [0, 0, 0, 0, 0, 0, 0, 0]
#                                   ]))
# Start , 1,2,3,4,5,6,7,8, End
df = pd.DataFrame(data = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                                   [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                                   [0, 0, 0, 0, 1, 0, 1, 0, 1, 0],
                                   [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
                                   [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
                                   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                  ]))

In [41]:
class Graph: 
   
    def __init__(self, vertices): 
        # No. of vertices 
        self.V = vertices  
          
        # default dictionary to store graph 
        self.graph = defaultdict(list)
        self.paths = []
   
    # function to add an edge to graph 
    
    def addEdge(self, u, v): 
        self.graph[u].append(v) 
   
    '''A recursive function to print all paths from 'u' to 'd'. 
    visited[] keeps track of vertices in current path. 
    path[] stores actual vertices and path_index is current 
    index in path[]'''
    def printAllPathsUtil(self, u, d, visited, path): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
  
        # If current vertex is same as destination, then print 
        # current path[] 
        if u == d: 
            print(f"[INFO] {path}")
            self.paths.append(path.copy())
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]== False: 
                    self.printAllPathsUtil(i, d, visited, path) 
                      
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop()
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self, s, d): 
  
        # Mark all the vertices as not visited 
        visited =[False]*(self.V) 
  
        # Create an array to store paths 
        path = [] 
  
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d, visited, path) 
        
def get_tenacious_schemes(paths, df):
    possible_schemes = []
    max_nodes = 0
    for path in paths:
        possible_schemes.append(path[1:-1])
        if max(path[1:-1]) > max_nodes:
            max_nodes = max(path[1:-1])
        
    all_nodes = list(range(1, max_nodes + 1))
    possible_schemes.append(all_nodes)
    
    for path in possible_schemes:
        xor_nodes = set(path).symmetric_difference(all_nodes)
        for i in range(1, len(xor_nodes)):
            combinations_xor_nodes = list(itertools.combinations(xor_nodes, i))
            for xor_node in combinations_xor_nodes:
                new_path = path.copy()
                new_path.extend(xor_node)
                new_path.sort()
                if new_path not in possible_schemes:
                    possible_schemes.append(new_path)
    return possible_schemes

def get_probabilities(shemes, probs):
    all_nodes = shemes[0].copy()
    P_states = []
    flag = False
    for path in shemes:
        if path == [1,2,3,4,5,7]:
            flag = True
        else:
            flag = False
        p_value = 1
        leftovers = all_nodes.copy()
        for node in path:
            p_value *= round(probs[node], 2)
            leftovers.remove(node)
            if flag:
                pass
        for broken_node in leftovers:
#             print(f'[INFO] broken node: {broken_node}')
            p_value *= round(1 - probs[broken_node], 2)
            if flag:
                pass
            #leftovers.remove(broken_node)
#         print('---')
        P_states.append(round(p_value, 6))
    return P_states

def get_lambda(probs, t):
    return - (np.log(sum(probs))/t)

def calculate_Lab2(df, P_list):
    g = Graph(df.shape[0])
    print(f"[NOTE] Node 0 and N+1 are the start and the end of the graph.")
    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            if df.iloc[i, j] == 1:
                if i == 0:
                    #g.addEdge("START", j)
                    g.addEdge(i, j)
                elif j == df.shape[0] - 1:
                    #g.addEdge(i, "END")
                    g.addEdge(i, j)
                else:
                    g.addEdge(i, j)
                print(f"[INFO] Added edge {i} -> {j}.")
    s = 0
    d = df.shape[0] - 1
    print("----------------------------------------------------------")
    print (f"[INFO] Following are all different paths from node {s} to {d} :") 
    g.printAllPaths(s, d)
    print("----------------------------------------------------------")
    
    schemes = get_tenacious_schemes(g.paths, df)
    schemes.sort(key=len)
    schemes = schemes[::-1]
    probs = get_probabilities(schemes, P_list)
    # probs.sort()
    # P_list = ["dummy", 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.92, 0.94]
    for i, j in zip(shemes, probs):
        print(f"[INFO] {i} = {j}")
    print("----------------------------------------------------------")
        
    P = round(sum(probs), 7)
    print(f"[INFO] P = {P}")
    
    lambda_value = get_lambda(probs, 10)
    T = 1/lambda_value
    print(f"[INFO] Lambda = {lambda_value}")
    print(f"[INFO] T = {T}")

In [42]:
calculate_Lab2(df, P_list)

[NOTE] Node 0 and N+1 are the start and the end of the graph.
[INFO] Added edge 0 -> 1.
[INFO] Added edge 1 -> 2.
[INFO] Added edge 1 -> 3.
[INFO] Added edge 2 -> 4.
[INFO] Added edge 2 -> 5.
[INFO] Added edge 3 -> 4.
[INFO] Added edge 3 -> 6.
[INFO] Added edge 3 -> 8.
[INFO] Added edge 4 -> 5.
[INFO] Added edge 4 -> 6.
[INFO] Added edge 4 -> 8.
[INFO] Added edge 5 -> 6.
[INFO] Added edge 5 -> 7.
[INFO] Added edge 6 -> 7.
[INFO] Added edge 6 -> 8.
[INFO] Added edge 7 -> 9.
[INFO] Added edge 8 -> 9.
----------------------------------------------------------
[INFO] Following are all different paths from node 0 to 9 :
[INFO] [0, 1, 2, 4, 5, 6, 7, 9]
[INFO] [0, 1, 2, 4, 5, 6, 8, 9]
[INFO] [0, 1, 2, 4, 5, 7, 9]
[INFO] [0, 1, 2, 4, 6, 7, 9]
[INFO] [0, 1, 2, 4, 6, 8, 9]
[INFO] [0, 1, 2, 4, 8, 9]
[INFO] [0, 1, 2, 5, 6, 7, 9]
[INFO] [0, 1, 2, 5, 6, 8, 9]
[INFO] [0, 1, 2, 5, 7, 9]
[INFO] [0, 1, 3, 4, 5, 6, 7, 9]
[INFO] [0, 1, 3, 4, 5, 6, 8, 9]
[INFO] [0, 1, 3, 4, 5, 7, 9]
[INFO] [0, 1, 3, 4, 6, 

In [30]:
g = Graph(df.shape[0])
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        if df.iloc[i, j] == 1:
            if i == 0:
                #g.addEdge("START", j)
                g.addEdge(i, j)
            elif j == df.shape[0] - 1:
                #g.addEdge(i, "END")
                g.addEdge(i, j)
            else:
                g.addEdge(i, j)
            #print(f"[INFO] Added edge {i} -> {j}.")

In [31]:
s = 0
d = df.shape[0] - 1
print (f"Following are all different paths from {s} to {d} :") 
g.printAllPaths(s, d)

Following are all different paths from 0 to 9 :
[INFO] [0, 1, 2, 4, 5, 6, 7, 9]
[INFO] [0, 1, 2, 4, 5, 6, 8, 9]
[INFO] [0, 1, 2, 4, 5, 7, 9]
[INFO] [0, 1, 2, 4, 6, 7, 9]
[INFO] [0, 1, 2, 4, 6, 8, 9]
[INFO] [0, 1, 2, 4, 8, 9]
[INFO] [0, 1, 2, 5, 6, 7, 9]
[INFO] [0, 1, 2, 5, 6, 8, 9]
[INFO] [0, 1, 2, 5, 7, 9]
[INFO] [0, 1, 3, 4, 5, 6, 7, 9]
[INFO] [0, 1, 3, 4, 5, 6, 8, 9]
[INFO] [0, 1, 3, 4, 5, 7, 9]
[INFO] [0, 1, 3, 4, 6, 7, 9]
[INFO] [0, 1, 3, 4, 6, 8, 9]
[INFO] [0, 1, 3, 4, 8, 9]
[INFO] [0, 1, 3, 6, 7, 9]
[INFO] [0, 1, 3, 6, 8, 9]
[INFO] [0, 1, 3, 8, 9]


In [32]:
shemes = get_tenacious_schemes(g.paths, df)
shemes.sort(key=len)
shemes = shemes[::-1]
probs = get_probabilities(shemes, P_list)
# probs.sort()
# P_list = ["dummy", 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.92, 0.94]
for i, j in zip(shemes, probs):
    print(f"[INFO] {i} = {j}")

[INFO] [1, 2, 3, 4, 5, 6, 7, 8] = 0.111144
[INFO] [1, 3, 4, 5, 6, 7, 8] = 0.074096
[INFO] [1, 2, 3, 5, 6, 7, 8] = 0.027786
[INFO] [1, 2, 3, 4, 6, 7, 8] = 0.019614
[INFO] [1, 2, 3, 4, 5, 7, 8] = 0.012349
[INFO] [1, 2, 3, 4, 5, 6, 8] = 0.009665
[INFO] [1, 2, 4, 5, 6, 7, 8] = 0.047633
[INFO] [1, 2, 3, 4, 5, 6, 7] = 0.007094
[INFO] [1, 3, 5, 6, 7, 8] = 0.018524
[INFO] [1, 2, 3, 6, 7, 8] = 0.004903
[INFO] [1, 3, 4, 6, 7, 8] = 0.013076
[INFO] [1, 3, 4, 5, 7, 8] = 0.008233
[INFO] [1, 2, 3, 5, 7, 8] = 0.003087
[INFO] [1, 2, 3, 5, 6, 8] = 0.002416
[INFO] [1, 2, 5, 6, 7, 8] = 0.011908
[INFO] [1, 2, 3, 5, 6, 7] = 0.001774
[INFO] [1, 2, 3, 4, 7, 8] = 0.002179
[INFO] [1, 2, 3, 4, 5, 8] = 0.001074
[INFO] [1, 2, 3, 4, 6, 8] = 0.001706
[INFO] [1, 2, 4, 6, 7, 8] = 0.008406
[INFO] [1, 2, 3, 4, 6, 7] = 0.001252
[INFO] [1, 2, 4, 5, 7, 8] = 0.005293
[INFO] [1, 2, 3, 4, 5, 7] = 0.000788
[INFO] [1, 3, 4, 5, 6, 8] = 0.006443
[INFO] [1, 3, 4, 5, 6, 7] = 0.00473
[INFO] [1, 2, 4, 5, 6, 8] = 0.004142
[INFO] [1, 2

In [25]:
round(sum(probs), 7)

0.434605

In [26]:
def get_lambda(probs, t):
    return - (np.log(sum(probs))/t)

lambda_value = get_lambda(probs, 10)
1/lambda_value

12.000225032146869